# Example Model for MM Data 
We will be training and evaluating a first order markov model (FOMM). 

In [4]:
import os, sys, datetime
sys.path.append('../')
from baseline_model import FOMM
from data import load_mmrf
from base import Model, setup_torch_dataset, pt_numpy
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt
import seaborn as sns
import torch
import torch.nn as nn
torch.manual_seed(0)
np.random.seed(0)

## Load in the Data

In [6]:
fold = 1 # specify which fold of mm data you want
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device  = torch.device('cpu')
print ('Running on: ', device)    
mmdata  = load_mmrf(fold_span = [fold], digitize_K = 20, digitize_method = 'uniform', suffix='_2mos')
dim_base, dim_data, dim_treat = mmdata[fold]['train']['b'].shape[-1], mmdata[fold]['train']['x'].shape[-1], mmdata[fold]['train']['a'].shape[-1]
train, train_loader = setup_torch_dataset(mmdata, fold, 'train', device)
valid, valid_loader = setup_torch_dataset(mmdata, fold, 'valid', device)
test, test_loader   = setup_torch_dataset(mmdata, fold, 'test', device)

Running on:  cuda
loading from: /afs/csail.mit.edu/u/z/zeshanmh/research/ml_mmrf/ml_mmrf_v1/cleaned_mm_fold_2mos.pkl
Digitizing outcomes ymax: 35.81666666666667
20 19 20
using digitized y
using digitized y
using digitized y


## Train and Evaluate Model

In [ ]:
dim_hidden = 300
model = FOMM(dim_hidden, dim_base, dim_data, dim_treat, mtype = 'linear', C =.1, reg_type = 'l1', reg_all = True)
model.to(device)
_, nelbo, nll, kl, _ = model.fit_unsupervised(train_loader, valid_loader, 15000, 1e-3, eval_freq=1, print_freq=1000, fname = 'test_model.pt') 

In [ ]:
model.eval()
(nelbo, nll, kl,_), _ = model.forward_unsupervised(*test_loader.dataset.tensors, anneal = 1.)
print(nll)